## Workbook of tonal harmony
#### Analysis of the string quartet Op. 59 n.3 mov. 1 (Introduzione) from music21

In [1]:
from pcsPy import *
import pickle, copy
from IPython.display import Image 
sys.path.append('../')
from tonalHarmonyDefs import enharmonicDictionary, shortHands, scoreAnalysis, tonalPartition, showAnalysis

<IPython.core.display.Javascript object>

Here we define the tonal space using the ABC database. A tonal space is the matrix of all the operators that connect the chords of a chord library as roman numerals.
Typical models are built using combinations of basic chord symbols and possible extension. 

In [37]:
scoretvs = '/Users/marco/Dropbox (Personal)/Musica/Applications/musicntwrk/RESULTS/MUSICOLOGY/Beethoven/ABC-1.0/data/tsv/op. 127 No. 12/op127_no12_mov1.tsv'

In [38]:
abc_dataset = pd.read_csv(scoretvs,sep='\t')

In [10]:
abc_dataset

,chord,altchord,measure,beat,totbeat,timesig,op,no,mov,length,global_key,local_key,pedal,numeral,form,figbass,changes,relativeroot,phraseend
0,.Eb.I,NaN,1,1.0,1.0,2/4,127,12,1,2.5,Eb,I,NaN,I,NaN,NaN,NaN,NaN,False
1,V43,NaN,2,1.5,3.5,2/4,127,12,1,1.5,Eb,I,NaN,V,NaN,43.0,NaN,NaN,False
2,I,NaN,3,1.0,5.0,2/4,127,12,1,2.5,Eb,I,NaN,I,NaN,NaN,NaN,NaN,False
3,V2,NaN,4,1.5,7.5,2/4,127,12,1,1.5,Eb,I,NaN,V,NaN,2.0,NaN,NaN,False
4,I6,NaN,5,1.0,9.0,2/4,127,12,1,1.0,Eb,I,NaN,I,NaN,6.0,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
553,I6(72),NaN,280,1.0,822.0,3/4,127,12,1,1.0,Eb,I,NaN,I,NaN,6.0,72,NaN,False
554,I6(7),NaN,280,2.0,823.0,3/4,127,12,1,1.0,Eb,I,NaN,I,NaN,6.0,7,NaN,False
555,I64,NaN,280,3.0,824.0,3/4,127,12,1,1.0,Eb,I,NaN,I,NaN,64.0,NaN,NaN,False
556,V7,NaN,281,1.0,825.0,3/4,127,12,1,3.0,Eb,I,NaN,V,NaN,7.0,NaN,NaN,False


In [34]:
rn = []
for n in range(len(abc_dataset)):
    if str(abc_dataset['form'][n]) != 'nan':
        roman = str(abc_dataset['numeral'][n])
        extension = str(abc_dataset['form'][n])
        if extension == '%':
            extension = '/o'
        if extension == 'M':
            extension = '7'
        if extension == 'Ger':
            extension = '7'
        rn.append(roman+extension)
    else:
        roman = str(abc_dataset['numeral'][n])
        if roman == 'nan':
            pass
        else:
            rn.append(roman)
rn = Remove(rn)

In [35]:
sorted(rn)

['#iv/o',
 '#vi',
 '#viio',
 'Ger',
 'I',
 'I+',
 'II',
 'III',
 'IV',
 'IV+',
 'IV7',
 'V',
 'VI',
 'VII',
 'i',
 'ii',
 'ii/o',
 'iii',
 'iio',
 'iv',
 'ivo',
 'v',
 'vi',
 'vii/o',
 'viio']

In [24]:
str(abc_dataset['form'][1])

'nan'

In [2]:
score = '/Users/marco/Dropbox (Personal)/Musica/Applications/musicntwrk/RESULTS/MUSICOLOGY/Beethoven/ABC-1.0/data/mscx/op. 127 No. 12/op127_no12_mov1.musicxml'

In [101]:
# score = 'Op59n3m1_Intro.musicxml'
seq,chords = readScore(score,music21=False,show=False)

In [ ]:
# To write chords and corresponding operators - uses musescore
WRITEscoreOps(seq,w=True)

In [4]:
dnodes,dedges,dcounts,deg,modul,Gx,Gxu = scoreNetwork(seq,ntx=True,general=True)
print('average degree = ',deg,' modularity = ',modul)

average degree =  3.247967479674797  modularity =  0.4332518980199036


In [98]:
# Write to csv for visualization in Gephi or other network drawing software (see figure below)
dnodes.to_csv('nodesOp127n12m1.csv')
dedges.to_csv('edgesOp127n12m1.csv')
# Get modularity scores
# part = cm.best_partition(Gxu) #,resolution=0.4)

In [73]:
def tonalPartition(seq,chords,nodes,Gx,Gxu,resolution=1.0,randomize=None,display=False):
    part = cm.best_partition(Gxu,resolution=resolution,randomize=randomize)
    dn = np.array(nodes)
    labe = []
    modu = []
    modul = []
    for m in range(len(dn)):
        labe.append(str(dn[m][0]))
        modu.append(part[str(m)])
        modul.append([str(dn[m][0]),Gx.degree()[str(m)],part[str(m)]])
    modul = pd.DataFrame(modul,columns=['Label','Degree','Modularity'])
    moduldict = dict(zip(labe,modu))

    if display:
    # display the score with modularity classes
        mc = []
        seqmc = []
        for n in range(len(seq)):
            p = PCSet(np.asarray(seq[n]))
            if p.pcs.shape[0] == 1:
                nn = ''.join(m21.chord.Chord(p.pcs.tolist()).pitchNames)
            else:
                nn = ''.join(m21.chord.Chord(p.normalOrder().tolist()).pitchNames)
            mc.append(nn)
            seqmc.append(moduldict[nn])

        l = 0
        analyzed = copy.deepcopy(chords)
        for c in analyzed.recurse().getElementsByClass('Chord'):
            c.closedPosition(forceOctave=4,inPlace=True)
            c.addLyric(str(l))
            c.addLyric(seqmc[l])
            l += 1
        analyzed.show('musicxml')
    return(moduldict,modul)

In [ ]:
moduldict,modul = tonalPartition(seq,chords,dnodes,Gx,Gxu,resolution=1.5,randomize=True,display=True)

In [87]:
modul.to_csv('modulOp127n12m1.csv')

In [83]:
modul[['Label','Degree','Modularity']].query('Modularity == 5').sort_values(by=['Degree'],ascending=False)

,Label,Degree,Modularity
218,G#BDD#,6,5
35,BDE-G,3,5
64,CDEG#,2,5
96,DE-A-,2,5
108,DEG#,2,5
212,G#B#DD#,2,5
215,G#BB#D#,2,5
217,G#BD#,2,5


In [84]:
# Dictionary of modularity classes and relative keys
keydict = {0:'E-', 1:'G', 2:'G', 3:'F', 4:'E-', 5:'E-'}

In [149]:
# manual control of modulations
keychange = None #{140:'E-',141:'E-'}
# manual control of roman numerals
altrn = None #{2:'II'}
# manual elimination of "unknown" chords
seq[40] = seq[39]
seq[44] = seq[43]
for i in range(54,63):
    seq[i] = seq[53]
seq[65] = [5,8,10,2]
for i in range(104,111):
    seq[i] = seq[103]
seq[138] = [5,8,0]

In [150]:
nxt,rn,ops = scoreAnalysis(seq,moduldict,keydict,keychange=keychange,altrn=altrn,table='../tonal.harmony.minimal')

modulation at or before chord no.  84
modulation at or before chord no.  88
modulation at or before chord no.  141
check error


In [151]:
analysis = showAnalysis(nxt,chords,seq,rn,ops,keydict,moduldict,last=False,display=False)

In [152]:
for i,n in enumerate(rn):
    print(i,n,ops[i],seq[i],chords[i])

0 I O(0,0,0) [3, 7, 10] <music21.metadata.Metadata object at 0x1a1e0a8bd0>
1 I O(2,1,0,-1) [3, 7, 10] P1: Violin I: Violin I (2)
2 V7 O(0,0,0,0) [2, 5, 8, 10] <music21.stream.Measure 1 offset=0.0>
3 V7 O(1,-2,-1,0) [2, 5, 8, 10] <music21.stream.Measure 2 offset=2.0>
4 I O(0,0,0) [3, 7, 10] <music21.stream.Measure 3 offset=4.0>
5 I O(2,1,0,-1) [3, 7, 10] <music21.stream.Measure 4 offset=6.0>
6 V7 O(0,0,0,0) [2, 5, 8, 10] <music21.stream.Measure 5 offset=8.0>
7 V7 O(1,-2,-1,0) [2, 5, 8, 10] <music21.stream.Measure 6 offset=10.0>
8 I O(0,0,2) [3, 7, 10] <music21.stream.Measure 7 offset=12.0>
9 vi O(0,0,1) [0, 3, 7] <music21.stream.Measure 8 offset=15.0>
10 IV O(0,0,0) [8, 0, 3] <music21.stream.Measure 9 offset=18.0>
11 IV O(0,0,0) [8, 0, 3] <music21.stream.Measure 10 offset=21.0>
12 IV O(0,0,0) [8, 0, 3] <music21.stream.Measure 11 offset=24.0>
13 IV O(0,0,0) [8, 0, 3] <music21.stream.Measure 12 offset=27.0>
14 IV O(0,0,0) [8, 0, 3] <music21.stream.Measure 13 offset=30.0>
15 IV O(0,0,0) [8

In [153]:
analysis

,pcs,chord,rn,ops,key,modul
0,"[3, 7, 10]",E-GB-,I,"O(0,0,0)",E-,0
1,"[3, 7, 10]",E-GB-,I,"O(2,1,0,-1)",E-,0
2,"[2, 5, 8, 10]",DFA-B-,V7,"O(0,0,0,0)",E-,0
3,"[2, 5, 8, 10]",DFA-B-,V7,"O(1,-2,-1,0)",E-,0
4,"[3, 7, 10]",E-GB-,I,"O(0,0,0)",E-,0
...,...,...,...,...,...,...
136,"[2, 5, 8, 11]",DFG#B,iio7,"O(1,-2,-1,1)",E-,0
137,"[0, 3, 7]",CE-G,i,"O(0,2,1)",E-,0
138,"[5, 8, 0]",FA-C,iv,"O(-2,-1,0)",E-,0
139,"[0, 3, 7]",CE-G,i,"O(-1,-1,0)",E-,0


In [154]:
seq[141]

[8, 0, 2]